In [1]:
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas as pd
import glob
import os
from joblib import Parallel, delayed
gc.collect()

0

In [19]:
df_0 = pd.read_csv('option.csv', encoding='cp949', nrows=4)

In [5]:
df_0 = pd.read_csv('spy_option.csv', encoding='cp949')

C:\Users\asas4\AppData\Local\Temp\ipykernel_3920\1142678970.py:1: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv('spy_option.csv', encoding='cp949')


In [6]:
df_0.shape

(21573704, 38)

In [34]:
import pandas as pd

chunks = pd.read_csv('option.csv', encoding='cp949', chunksize=1000000, low_memory=False)

for i, chunk in enumerate(chunks):
    chunk.to_parquet(
        f'chunks/option_chunk_{i}.parquet',
        engine='pyarrow',
        index=False,
    )

In [27]:
len(df_0.select_dtypes(include=[np.number]).columns.tolist())

26

In [33]:
cat_col = [col for col in df_0.columns if col not in df_0.select_dtypes(include=[np.number]).columns.tolist()]

In [36]:
cat_col

['date',
 'symbol',
 'exdate',
 'last_date',
 'cp_flag',
 'root',
 'suffix',
 'ticker',
 'issue_type',
 'issuer',
 'div_convention',
 'exercise_style']

In [35]:
df_0[cat_col]

,date,symbol,exdate,last_date,cp_flag,root,suffix,ticker,issue_type,issuer,div_convention,exercise_style
0,1996-01-04,098D5.5F,1996-01-20,1996-01-04,C,098D5,5F,XMI,A,NYSE ARCA MAJOR MARKET,I,E
1,1996-01-04,09931.91,1996-01-20,NaN,C,09931,91,XMI,A,NYSE ARCA MAJOR MARKET,I,E
2,1996-01-04,09954.7D,1996-02-17,NaN,C,09954,7D,XMI,A,NYSE ARCA MAJOR MARKET,I,E
3,1996-01-04,09A02.DC,1996-03-16,NaN,C,09A02,DC,XMI,A,NYSE ARCA MAJOR MARKET,I,E


In [39]:
np.unique(df_0['exercise_style'])

array(['E'], dtype=object)

In [25]:
df_ = pd.read_csv('option.csv', encoding='cp949', usecols=['ticker', 'issuer'])

In [28]:
df_.shape[0]//1000000

107

In [40]:
df_2 = pd.read_csv('option.csv', encoding='cp949', usecols=['cp_flag', 'exercise_style', 'issue_type'])

C:\Users\asas4\AppData\Local\Temp\ipykernel_21012\3733477512.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv('option.csv', encoding='cp949', usecols=['cp_flag', 'exercise_style', 'issue_type'])


In [17]:
np.unique(df_['issuer'])

array(['APPLE INC', 'CBOE MINI-NDX INDEX', 'CBOE S&P 100 INDEX',
       'CBOE S&P 500 INDEX', 'DOW JONES INDL AVG INDEX',
       'JOHNSON & JOHNSON', 'MASTERCARD INCORPORATED',
       'META PLATFORMS INC', 'MICROSOFT CORPORATION', 'NASDAQ 100 INDEX',
       'NIKE, INC.', 'NVIDIA CORP', 'NYSE ARCA MAJOR MARKET'],
      dtype=object)

In [41]:
np.unique(df_2['cp_flag'])

array(['C', 'P'], dtype=object)

In [42]:
np.unique(df_2['exercise_style'])

array(['A', 'E'], dtype=object)

In [44]:
df_2['issue_type'].unique()

array(['A', '0', 0], dtype=object)

In [46]:
list(df_['ticker'].unique())

['XMI',
 'AAPL',
 'NDX',
 'JNJ',
 'MSFT',
 'SPX',
 'NKE',
 'OEX',
 'DJX',
 'NVDA',
 'MNX',
 'MA',
 'META']

In [35]:
# 필터링 대상
tickers = ['XMI', 'AAPL', 'NDX', 'JNJ', 'MSFT', 'SPX', 'NKE', 'OEX', 'DJX', 'NVDA', 'MNX', 'MA', 'META']
cp_flags = ['C', 'P']
exercise_styles = ['A', 'E']

# 모든 조합 생성
jobs = [(ticker, cp, style) for ticker in tickers for cp in cp_flags for style in exercise_styles]

In [55]:
jobs

[('XMI', 'C', 'A'),
 ('XMI', 'C', 'E'),
 ('XMI', 'P', 'A'),
 ('XMI', 'P', 'E'),
 ('AAPL', 'C', 'A'),
 ('AAPL', 'C', 'E'),
 ('AAPL', 'P', 'A'),
 ('AAPL', 'P', 'E'),
 ('NDX', 'C', 'A'),
 ('NDX', 'C', 'E'),
 ('NDX', 'P', 'A'),
 ('NDX', 'P', 'E'),
 ('JNJ', 'C', 'A'),
 ('JNJ', 'C', 'E'),
 ('JNJ', 'P', 'A'),
 ('JNJ', 'P', 'E'),
 ('MSFT', 'C', 'A'),
 ('MSFT', 'C', 'E'),
 ('MSFT', 'P', 'A'),
 ('MSFT', 'P', 'E'),
 ('SPX', 'C', 'A'),
 ('SPX', 'C', 'E'),
 ('SPX', 'P', 'A'),
 ('SPX', 'P', 'E'),
 ('NKE', 'C', 'A'),
 ('NKE', 'C', 'E'),
 ('NKE', 'P', 'A'),
 ('NKE', 'P', 'E'),
 ('OEX', 'C', 'A'),
 ('OEX', 'C', 'E'),
 ('OEX', 'P', 'A'),
 ('OEX', 'P', 'E'),
 ('DJX', 'C', 'A'),
 ('DJX', 'C', 'E'),
 ('DJX', 'P', 'A'),
 ('DJX', 'P', 'E'),
 ('NVDA', 'C', 'A'),
 ('NVDA', 'C', 'E'),
 ('NVDA', 'P', 'A'),
 ('NVDA', 'P', 'E'),
 ('MNX', 'C', 'A'),
 ('MNX', 'C', 'E'),
 ('MNX', 'P', 'A'),
 ('MNX', 'P', 'E'),
 ('MA', 'C', 'A'),
 ('MA', 'C', 'E'),
 ('MA', 'P', 'A'),
 ('MA', 'P', 'E'),
 ('META', 'C', 'A'),
 ('META', '

In [46]:


# 설정
input_dir = 'chunks'
output_dir = 'filtered'
os.makedirs(output_dir, exist_ok=True)


# 청크 파일 목록 가져오기
chunk_files = sorted(glob.glob(f'{input_dir}/option_chunk_*.parquet'))

filtered_parts = []

for condition in jobs:
    ticker, cp_flag, exercise_style = condition
    filters = [
        ('ticker', '==', ticker),
        ('cp_flag', '==', cp_flag),
        ('exercise_style', '==', exercise_style)
    ]
    
    for file in chunk_files:
        try:
            # 필터 조건에 맞는 데이터만 읽기
            df = pd.read_parquet(file, filters=filters, engine='pyarrow')  # pyarrow 필수
            if not df.empty:
                
                filtered_parts.append(df)
                del df
                gc.collect()

        except Exception as e:
            print(f"Skipping {file} due to: {e}")

    if filtered_parts:
        # 결과를 하나의 DataFrame으로 합치기
        result_df = pd.concat(filtered_parts, ignore_index=True)
        
        # 결과를 Parquet 파일로 저장
        output_file = f'{output_dir}/{ticker}_{cp_flag}_{exercise_style}.parquet'
        # 문자열 기반 object 컬럼만 category로 변환
        for col in result_df.select_dtypes(include='object').columns:
            if result_df[col].map(type).eq(str).all():
                result_df[col] = result_df[col].astype('category')
            else:
                result_df[col] = result_df[col].astype(str)  # fallback

        result_df.to_parquet(output_file, engine='pyarrow', index=False)
        
        print(f"Saved filtered data to {output_file}")
        del result_df
        gc.collect()
        
        filtered_parts = []

Saved filtered data to filtered/XMI_C_E.parquet
Saved filtered data to filtered/XMI_P_E.parquet
Saved filtered data to filtered/AAPL_C_A.parquet
Saved filtered data to filtered/AAPL_P_A.parquet
Saved filtered data to filtered/NDX_C_E.parquet
Saved filtered data to filtered/NDX_P_E.parquet
Saved filtered data to filtered/JNJ_C_A.parquet
Saved filtered data to filtered/JNJ_P_A.parquet
Saved filtered data to filtered/MSFT_C_A.parquet
Saved filtered data to filtered/MSFT_P_A.parquet
Saved filtered data to filtered/SPX_C_E.parquet
Saved filtered data to filtered/SPX_P_E.parquet
Saved filtered data to filtered/NKE_C_A.parquet
Saved filtered data to filtered/NKE_P_A.parquet
Saved filtered data to filtered/OEX_C_A.parquet
Saved filtered data to filtered/OEX_P_A.parquet
Saved filtered data to filtered/DJX_C_E.parquet
Saved filtered data to filtered/DJX_P_E.parquet
Saved filtered data to filtered/NVDA_C_A.parquet
Saved filtered data to filtered/NVDA_P_A.parquet
Saved filtered data to filtered/MN

In [47]:
check_df = pd.read_parquet('filtered/AAPL_C_A.parquet', engine='pyarrow')

In [53]:
check_df.columns

Index(['secid', 'date', 'symbol', 'symbol_flag', 'exdate', 'last_date',
       'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume',
       'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta',
       'optionid', 'cfadj', 'am_settlement', 'contract_size', 'ss_flag',
       'forward_price', 'expiry_indicator', 'root', 'suffix', 'cusip',
       'ticker', 'sic', 'index_flag', 'exchange_d', 'class', 'issue_type',
       'industry_group', 'issuer', 'div_convention', 'exercise_style',
       'am_set_flag'],
      dtype='object')

In [7]:
df_0.head()

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,109820,2005-01-10,SFB.AA,0,2005-01-22,NaN,C,131000,0.0,0.05,...,NaN,0,1,NaN,%,NaN,SSGA ACTIVE TRUST,NaN,A,NaN
1,109820,2005-01-10,SFB.AB,0,2005-01-22,NaN,C,132000,0.0,0.05,...,NaN,0,1,NaN,%,NaN,SSGA ACTIVE TRUST,NaN,A,NaN
2,109820,2005-01-10,SFB.AC,0,2005-01-22,NaN,C,133000,0.0,0.05,...,NaN,0,1,NaN,%,NaN,SSGA ACTIVE TRUST,NaN,A,NaN
3,109820,2005-01-10,SFB.AD,0,2005-01-22,NaN,C,134000,0.0,0.05,...,NaN,0,1,NaN,%,NaN,SSGA ACTIVE TRUST,NaN,A,NaN
4,109820,2005-01-10,SFB.AE,0,2005-01-22,NaN,C,135000,0.0,0.05,...,NaN,0,1,NaN,%,NaN,SSGA ACTIVE TRUST,NaN,A,NaN


In [9]:
np.unique(df_0['cp_flag'])

array(['C', 'P'], dtype=object)

In [10]:
df_0[df_0['cp_flag'] == 'C'].to_parquet('filtered/spy_C_A.parquet', engine='pyarrow', index=False)

In [11]:
df_0[df_0['cp_flag'] == 'P'].to_parquet('filtered/spy_P_A.parquet', engine='pyarrow', index=False)

In [2]:
os.listdir('filtered')

['MA_P_A.parquet',
 'AAPL_P_A.parquet',
 'JNJ_C_A.parquet',
 'AAPL_C_A.parquet',
 'MNX_P_E.parquet',
 'MSFT_P_A.parquet',
 'SPX_P_E.parquet',
 'NKE_P_A.parquet',
 'NDX_P_E.parquet',
 'MNX_C_E.parquet',
 'JNJ_P_A.parquet',
 'OEX_C_A.parquet',
 'NKE_C_A.parquet',
 'NVDA_C_A.parquet',
 'spy_P_A.parquet',
 'MA_C_A.parquet',
 'DJX_C_E.parquet',
 'NDX_C_E.parquet',
 'XMI_C_E.parquet',
 'MSFT_C_A.parquet',
 'spy_C_A.parquet',
 'SPX_C_E.parquet',
 'XMI_P_E.parquet',
 'DJX_P_E.parquet',
 'NVDA_P_A.parquet',
 'META_P_A.parquet',
 'OEX_P_A.parquet',
 'META_C_A.parquet']